In [1]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import normalize
from pathlib import Path
import matplotlib.pyplot as plt
import scipy

In [3]:
channels_to_skip = ['MKR1+', 'MKR2+', 'EKG+']
path = Path('sentences_data/sentences/')
outPath = Path('./eeg/')
patient_data = {}
i=0

for file in path.rglob('*sentences.xdf'):

    patient_id, session_id = file.name.split("_")[:2]
    if i == 1:
         break
    if session_id != "ses1":
         continue

    print(f"Patient ID: {patient_id}, session: {session_id}")
    eeg_ts = np.load(f"sentences_data/eeg/{patient_id}_{session_id}_sentences_sEEG.npy")
    audio_ts = np.load(f"sentences_data/eeg/{patient_id}_{session_id}_sentences_audio.npy")
    ch_names = np.load(f"sentences_data/eeg/{patient_id}_{session_id}_sentences_channelNames.npy")
    sentences_ts = np.load(f"sentences_data/eeg/{patient_id}_{session_id}_sentences.npy")

    channels_to_skip_idx = [np.where(ch==ch_names)[0][0] for ch in channels_to_skip]
    ch_names = np.delete(ch_names, channels_to_skip_idx)
    eeg_ts = np.delete(eeg_ts, channels_to_skip_idx, axis=1)
    
    patient_data[(patient_id, session_id)] = {
         "eeg":eeg_ts,
         "audio":audio_ts,
         "channels":ch_names,
         "sentences":sentences_ts
    }

    i+=1

Patient ID: p00, session: ses1


In [5]:
sent_dictionary = {}
for k, v in patient_data.items():
    unique_sent = np.unique(v["sentences"])
    for sent in unique_sent:
        if sent in sent_dictionary.keys():
            sent_dictionary[sent] += 1
        else:
            sent_dictionary[sent] = 1
    patient_data[k]["sentenceList"] = unique_sent

In [6]:
for patient_key, data in patient_data.items():
    # get unique sentences 
    sentences, sentences_start_idx, sentences_ts_indexed = np.unique(data["sentences"], return_index=True, return_inverse=True)

    # get sentences timepoints
    sentences_pos_dict = {i:{"sentence":sent, "indicies":[]} for i, sent in enumerate(sentences)}
    prev_sent_key = sentences_ts_indexed[0]
    start_pos_i = 0 

    for pos_i, current_sent_key in enumerate(sentences_ts_indexed):
        if current_sent_key != prev_sent_key:
            idx_tuple = (start_pos_i, pos_i)
            sentences_pos_dict[prev_sent_key]["indicies"].append(idx_tuple)
            start_pos_i = pos_i
        prev_sent_key = current_sent_key
    
    patient_data[patient_key]["sentencesPositions"] = sentences_pos_dict

In [7]:
[(k,v) for k,v in sent_dictionary.items() if v > 3]

[]

In [8]:
from scipy.fft import fft, fftfreq, rfft, rfftfreq

sentence_key = [(k, v) for k, v in sent_dictionary.items() if v > 2][1][0]

i=0
fig, axs = plt.subplots(3, 2, figsize=(15,10))
axs = axs.reshape(-1)
for pt, pt_data in patient_data.items():
    if sentence_key in pt_data["sentenceList"]:
        sentence_idx = np.where(pt_data["sentenceList"]==sentence_key)[0][0]
        sentence_position = pt_data["sentencesPositions"][sentence_idx]["indicies"]
        eeg = pt_data["eeg"][sentence_position[0][0]:sentence_position[0][1],:]
        eeg_mean = np.mean(eeg, axis=1)

        yf = rfft(eeg_mean)
        xf = rfftfreq(len(eeg_mean), 1 / 1024)

        # axs[i].plot(xf, np.abs(yf))
        axs[i].plot(eeg_mean)
        i+=1



IndexError: list index out of range

In [ ]:
for sent_key in range(0, len(sentences))[:1]:
    pos_tup_arr = sentences_dict[sent_key]["indicies"]
    sentence_str = sentences_dict[sent_key]["sentence"]
    print(f"Sentence: {sentence_str}")

    for pos_tup in pos_tup_arr[1:2]:
        eeg_sent_ts = eeg_ts[pos_tup[0]:pos_tup[1],40:50]
        plt.plot(eeg_sent_ts)
        plt.show()

In [25]:
import scipy 

scipy.stats.zscore(list(patient_data.values())[0]["eeg"]).shape


(512475, 107)